# DF

In [24]:
%store -r df_one_hot

# PACKAGES

In [25]:
# package that we need
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import MeanAbsolutePercentageError
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,accuracy_score, precision_score, recall_score,f1_score
from sklearn.model_selection import KFold

# essential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#correlation
import seaborn as sns

#! pip install pydot

# FEATURE AND TARGET

In [26]:
numerical_features = [
    'spotify_track_duration_minute', 'danceability', 'energy', 'loudness',
    'mode', 'speechiness', 'acousticness',
    'spotify_track_popularity', 'instrumentalness', 'liveness']

genre_features = [col for col in df_one_hot.columns if col.startswith('genre_')]

# UNION OF THE TWO 
total_features = numerical_features + genre_features

target='valence'

# PLOT

interesting plots for visualize the output of our model

In [27]:
def plot_for_model(history):

    #plot loss
    plt.figure()
    plt.plot(history.history['loss'],color='blue',label='loss')
    plt.plot(history.history['val_loss'],color='red',label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss function through epoch')
    plt.legend()
    plt.grid()
    plt.show()

    #plot mean
    plt.figure()
    plt.plot(history.history['mae'],color='blue',label='mae')
    plt.plot(history.history['val_mae'],color='red',label='val_mae')
    plt.xlabel('Epoch')
    plt.ylabel('mae')
    plt.title('Mae function through epoch')
    plt.grid()
    plt.legend()
    plt.show()   

# SPLIT DATA

In [28]:
def return_train_test_for_cross(train_index, test_index, df):
    #extract the feature and target
    x = df[total_features]
    y = df[target]

    #normalization
    scaler = StandardScaler()
    x[numerical_features] = scaler.fit_transform(x[numerical_features])
    #display(x)

    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    #display(x_train)

    return x_train,x_test,y_train,y_test

# MODEL

In [29]:
# function to create the model
def create_model_user(shape_x_train): #number of columns -> shape
    
    
    '''model_user_i = Sequential([
        Dense(32, input_dim=shape_x_train, activation='relu', name='first_layer'),
        Dense(16, activation='relu', name='secod_layer'),
        Dense(1, activation='sigmoid', name='final_layer')
    ])'''

    model_user_i = Sequential([
        
        Dense(32,input_dim=shape_x_train,activation='relu',name='first_layer',kernel_regularizer=l2(0.01)),
        Dropout(0.3),
        Dense(16,activation='relu',name='secod_layer',kernel_regularizer=l2(0.01)),
        Dropout(0.2),
        #Dense(1,activation='linear',name='final_layer')
        Dense(1,activation='sigmoid',name='final_layer') 
    ])

    #compile the model
    model_user_i.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae'])

    #summary
    #display(model_user_i.summary())

    #return  the model
    return model_user_i
    

# FIT MODEL

In [30]:
#function to train the model
def fit_model(model, x_train, y_train, x_test, y_test):

    history_model = model.fit(
        x_train,y_train,
        validation_data = [x_test, y_test],
        #epochs = 1,
        epochs=100,
        batch_size=32,
        #verbose=1
    )

    return history_model

# RECOMANDATION FUNCTION + CONFUSION MATRIX

In [31]:
def reco_function_conf_matrix(y_test, y_hat, threshold):
    
    #initialize dataframe    
    data = {'Original Test': y_test,
        'Prediction Test': y_hat.flatten(),
        }
    df_prediction = pd.DataFrame(data = data)
    df_prediction['Original over Thrashold'] = df_prediction['Original Test'] > threshold
    df_prediction['Predicted over Thrashold'] = df_prediction['Prediction Test'] > threshold
    #display(df_prediction)

    #plot confusion matrix of reccomandation
    cm = confusion_matrix(df_prediction['Original over Thrashold'], df_prediction['Predicted over Thrashold'])

    # METRIX FOR EVALUATE RECCOMANDATION 
    # Extract TP, TN, FP, FN
    TN, FP, FN, TP = cm.ravel()

    # Calculate metrics manually
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    '''plt.figure()
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['True', 'False'], yticklabels=['True', 'False'])
    plt.title('Confiusion matrix')
    plt.xlabel('predicted')
    plt.ylabel('actual')
    plt.show()'''

    # Store metrics in a DataFrame
    df_metrics_cm = pd.DataFrame(
        data =
        #'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-score'],
        {'Value': [accuracy, precision, recall, f1]},
        index = ['Accuracy', 'Precision', 'Recall', 'F1-score']
    )

    #display(df_metrics_cm.round(2))

    return df_prediction,df_metrics_cm

# MAIN 

In [32]:
dizio_model_user = {} #in this dizio we store the model for each user
dizio_accuracy = {} #accuracy of the model

# DICTIONARY TO STORE SONG RECOMMENDATION PER USER
reccomandations_songs_per_user={}

#threshold
threshold = 0.7

#value k for k-fold
k = 5

#loop creation user and model
for i in range(0, len(df_one_hot['User'].unique())): #i indica lo user   

    print('---------------------------------------------------------')
    print(f'USER {i}')

    #def dizio
    df_metrics_kfold = pd.DataFrame()

    #select user
    mask_user = df_one_hot['User'] == i
    #display(mask_user)
    user_i_df = df_one_hot[mask_user]
    #display(user_i_df) 

    #cross-validation k-fold
    kf = KFold(n_splits = k, shuffle= True, random_state= 42)
    score_fold = []

    #loop on the k-th folder
    loop = 0
    for train_index, test_index in kf.split(user_i_df):
        
        print('---------------------------------------------------------')
        print(f'K {loop} FOLD') 
        loop = loop + 1

        #return train and test
        x_train,x_test,y_train,y_test = return_train_test_for_cross(train_index, test_index, user_i_df) #da modificare

        #creation model for the specific user
        model_user_i = create_model_user(x_train.shape[1])
        
        #fit the model with the different x, y train (for different user)
        history_model_user_i = fit_model(model_user_i, x_train, y_train, x_test, y_test)
        
        #prediction
        y_hat = model_user_i.predict(x_test)

        #plot the metrics of the model
        #plot_for_model(history_model_user_i)

        #accuracy
        test_loss, test_mae = model_user_i.evaluate(x_test,y_test,verbose=1)
        #print('MSE: ', test_loss)
        #print('MAE:', test_mae)

        score_fold.append(test_mae)

        #reccomandation
        reccomandation_df = reco_function_conf_matrix(y_test, y_hat, threshold)
        df_metrics_kfold = pd.concat([df_metrics_kfold,reccomandation_df[1]], axis = 1)

    #save the model in the dictionary
    #add also the raccomandations
    dizio_model_user[f"User{i}"] = {#'model_user_i' : model_user_i,
                                    #'history_model_user_i': history_model_user_i,
                                    #'x_train': x_train,
                                    #'x_test': x_test,
                                    #'y_train': y_train,
                                    #'y_test': y_test,
                                    #'y_hat': y_hat,
                                    #'recommendations:': reccomandation_df
                                    'score_fold': score_fold,
                                    'df_metrics_kfold_mean': pd.DataFrame(data = df_metrics_kfold.mean(axis=1), columns=['Mean_Metrics'])
                                    } 
    #save accuracy 
    dizio_accuracy[f'User{i}'] = {
        #'MAE' : mean_absolute_error(y_test, y_hat),
        #'MSE' : mean_squared_error(y_test, y_hat)
        'Mean MAE': np.mean(score_fold)
    }

---------------------------------------------------------
USER 0
---------------------------------------------------------
K 0 FOLD


C:\Users\elped\AppData\Local\Temp\ipykernel_15892\330544952.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[numerical_features] = scaler.fit_transform(x[numerical_features])
c:\Users\elped\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.4411 - mae: 0.1950 - val_loss: 0.1857 - val_mae: 0.1662
Epoch 2/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1511 - mae: 0.1713 - val_loss: 0.0747 - val_mae: 0.1560
Epoch 3/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0676 - mae: 0.1615 - val_loss: 0.0460 - val_mae: 0.1475
Epoch 4/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0460 - mae: 0.1530 - val_loss: 0.0393 - val_mae: 0.1450
Epoch 5/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0414 - mae: 0.1519 - val_loss: 0.0376 - val_mae: 0.1440
Epoch 6/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0398 - mae: 0.1483 - val_loss: 0.0369 - val_mae: 0.1433
Epoch 7/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0396 - mae: 0.1482 - val_loss: 0.0365 - val_mae: 0.1439
Epoch 8/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0390 - mae: 0.1499 - val_loss: 0.0364 - val_mae: 0.1417
Epoch 9/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/

C:\Users\elped\AppData\Local\Temp\ipykernel_15892\330544952.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[numerical_features] = scaler.fit_transform(x[numerical_features])
c:\Users\elped\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


143/143 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.4904 - mae: 0.2549 - val_loss: 0.2016 - val_mae: 0.1752
Epoch 2/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1626 - mae: 0.1724 - val_loss: 0.0860 - val_mae: 0.1703
Epoch 3/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0754 - mae: 0.1683 - val_loss: 0.0545 - val_mae: 0.1640
Epoch 4/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0530 - mae: 0.1647 - val_loss: 0.0456 - val_mae: 0.1587
Epoch 5/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0442 - mae: 0.1566 - val_loss: 0.0425 - val_mae: 0.1557
Epoch 6/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0423 - mae: 0.1549 - val_loss: 0.0416 - val_mae: 0.1536
Epoch 7/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0407 - mae: 0.1526 - val_loss: 0.0406 - val_mae: 0.1519
Epoch 8/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0412 - mae: 0.1531 - val_loss: 0.0401 - val_mae: 0.1515
Epoch 9/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss:

...

C:\Users\elped\AppData\Local\Temp\ipykernel_15892\330544952.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[numerical_features] = scaler.fit_transform(x[numerical_features])
c:\Users\elped\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


143/143 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.4555 - mae: 0.2148 - val_loss: 0.1946 - val_mae: 0.1802
Epoch 2/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1567 - mae: 0.1812 - val_loss: 0.0793 - val_mae: 0.1678
Epoch 3/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0686 - mae: 0.1646 - val_loss: 0.0488 - val_mae: 0.1575
Epoch 4/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0466 - mae: 0.1572 - val_loss: 0.0417 - val_mae: 0.1528
Epoch 5/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0412 - mae: 0.1511 - val_loss: 0.0400 - val_mae: 0.1518
Epoch 6/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0414 - mae: 0.1547 - val_loss: 0.0391 - val_mae: 0.1502
Epoch 7/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0407 - mae: 0.1533 - val_loss: 0.0388 - val_mae: 0.1496
Epoch 8/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0390 - mae: 0.1501 - val_loss: 0.0381 - val_mae: 0.1474
Epoch 9/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss:

C:\Users\elped\AppData\Local\Temp\ipykernel_15892\330544952.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[numerical_features] = scaler.fit_transform(x[numerical_features])
c:\Users\elped\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


117/117 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.4754 - mae: 0.2199 - val_loss: 0.2399 - val_mae: 0.2015
Epoch 2/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2015 - mae: 0.2038 - val_loss: 0.1082 - val_mae: 0.1789
Epoch 3/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0936 - mae: 0.1753 - val_loss: 0.0615 - val_mae: 0.1591
Epoch 4/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0603 - mae: 0.1657 - val_loss: 0.0474 - val_mae: 0.1516
Epoch 5/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0482 - mae: 0.1565 - val_loss: 0.0437 - val_mae: 0.1520
Epoch 6/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0468 - mae: 0.1598 - val_loss: 0.0421 - val_mae: 0.1476
Epoch 7/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0432 - mae: 0.1517 - val_loss: 0.0415 - val_mae: 0.1475
Epoch 8/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0436 - mae: 0.1532 - val_loss: 0.0401 - val_mae: 0.1456
Epoch 9/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss:

KeyboardInterrupt: 

# DISPLAY DICTIONARY ACCURAY

In [ ]:
display(dizio_accuracy)

{'User0': {'Mean MAE': 0.13768224120140077},
 'User1': {'Mean MAE': 0.13333794772624968},
 'User2': {'Mean MAE': 0.09675337672233582},
 'User3': {'Mean MAE': 0.13251740336418152},
 'User4': {'Mean MAE': 0.13890121281147003}}

# DISPLAY DF METRICS CONFUSION MATRIX

In [ ]:
dizio_model_user['User4']['df_metrics_kfold_mean']

,Mean_Metrics
Accuracy,0.638398
Precision,0.708271
Recall,0.030630
F1-score,0.057896


In [ ]:
df_metrics_total = pd.concat([dizio_model_user['User0']['df_metrics_kfold_mean'],dizio_model_user['User1']['df_metrics_kfold_mean'],dizio_model_user['User2']['df_metrics_kfold_mean'],dizio_model_user['User3']['df_metrics_kfold_mean'],dizio_model_user['User4']['df_metrics_kfold_mean']], axis=1)
#type(df_metrics_total)
#df_metrics_total.columns(colume_name = ['User0','User1','User2','User3','User4'])
df_metrics_total.columns = ['User0','User1','User2','User3','User4']
display(df_metrics_total)

,User0,User1,User2,User3,User4
Accuracy,0.614100,0.628623,0.737348,0.760926,0.638398
Precision,0.740556,0.657268,0.852976,0.400000,0.708271
Recall,0.023681,0.048072,0.260070,0.002398,0.030630
F1-score,0.044802,0.084327,0.391596,0.004766,0.057896


In [ ]:
df_metrics_mean = pd.DataFrame(data = df_metrics_total.mean(axis=1), columns=['Mean_Metrics'])
display(df_metrics_mean.round(3))

,Mean_Metrics
Accuracy,0.675879
Precision,0.671814
Recall,0.072970
F1-score,0.116677
